In [1]:
import math
import random
import os
import time
import json
from functools import partial
from collections import defaultdict

import numpy as np
import pandas as pd

%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt

import tensorflow as tf

### Check Version

In [2]:
tf.__version__

'2.9.0'

### GPU Configuration

In [3]:
gpus = tf.config.list_physical_devices('GPU')

In [4]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
tf.config.experimental.set_memory_growth(gpus[0], True)

### Pandas Config

In [6]:
pd.set_option('display.max_column', None)

### Input Pipeline

In [7]:
IMG_WIDTH = 256
IMG_HEIGHT = 256

TRAIN_BATCH_SIZE = 4
VAL_BATCH_SIZE = 4

In [8]:
CROP_WIDTH = 227
CROP_HEIGHT = 227

In [9]:
foldFiles = ["adience/unprocessed/fold_0_data.txt",
             "adience/unprocessed/fold_1_data.txt",
             "adience/unprocessed/fold_2_data.txt",
             "adience/unprocessed/fold_3_data.txt",
             "adience/unprocessed/fold_4_data.txt"]

In [10]:
genderMap = defaultdict(lambda : np.NaN)
genderMap['m'] = 0
genderMap['f'] = 1

In [11]:
ages = ['(0, 2)', '(4, 6)', '(8, 13)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)']
ageMap = defaultdict(lambda : np.NaN)
for i,a in enumerate(ages):
    ageMap[a] = i

### Dataset Generation

Dataset generation pipeline:
Input: 
- foldFile - Path to fold file. Fold file Describes which images are in the fold and its corresponding labels
- imgBaseFolder - Base folder to search image from
- imgPrefix - Prefix of image file
- genderMap - Map from ['m', 'f', 'u', None], to one hot index
- ageMap - Map from age category to one hot index
- imgWidth - Resulting image width
- imgHeigh - Resulting image height
- batchSize - Int or None, batch size
- configureDs - Function accepting dataset for performance configurations
- preBatch - List of (name, functions) pair that will be mapped before batching. name is used as name parameters for tf graph 
- postBatch - List of (name, functions) that will be mapped after batching. name is used as name parameters for tf graph

The processing functions should have signature function(img, label) -> (img, label)

Pipeline

Read Fold File -> Preprocess filename and labels (dataframe) -> Convert filename and labels to numpy array -> Convert filename and labels to tf dataset -> Parse images and labels -> Configure Dataset for performance -> Pre-Batching preprocessing -> Batch -> Post-Batching preprocessing -> Output

Some preprocessing steps can only be done before and some can only be done after batching, thats why there are seperated pre and post batching list

In [12]:
def generateDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, batchSize, configureDs=None, preBatch=[], postBatch=[]):
    
    def parseImage(filename):
        img = tf.io.read_file(filename)
        img = tf.image.decode_jpeg(img)
        img = tf.image.resize(img, [imgHeight, imgWidth])
         
        return img
    
    # Read Fold File
    foldData = []
    for f in foldFiles:
        foldData.append(pd.read_csv(f, sep="\t"))
    foldData = pd.concat(foldData)
    
    # Form File Name
    foldData['filename'] = foldData.apply(lambda r: os.path.join(imgBaseFolder, r['user_id'], f"{imgPrefix}.{r['face_id']}.{r['original_image']}"), axis=1)
    
    # Generate Label One Hot Index
    foldData['gender_ind'] = foldData['gender'].map(genderMap)
    foldData['age_ind'] = foldData['age'].map(ageMap)
    
    # Remove dirty data
    foldData.dropna(subset=['gender_ind', 'age_ind'], inplace=True)
    
    # Dataframe to numpy
    filenames = foldData['filename'].to_numpy()
    
    genderIndex = foldData['gender_ind'].to_numpy().astype(int)
    ageIndex = foldData['age_ind'].to_numpy().astype(int)
    
    # Numpy to Dataset
    fnDs = tf.data.Dataset.from_tensor_slices(filenames)
    genderIndDs = tf.data.Dataset.from_tensor_slices(genderIndex)
    ageIndDs = tf.data.Dataset.from_tensor_slices(ageIndex)
    
    # Parse Images
    imageDs = fnDs.map(parseImage, num_parallel_calls=tf.data.AUTOTUNE, name="parse_image")
    
    # Parse Labels
    genderLabDs = genderIndDs.map(lambda x: tf.one_hot(x, genderDepth), num_parallel_calls=tf.data.AUTOTUNE, name="gender_one_hot")
    ageLabDs = ageIndDs.map(lambda x: tf.one_hot(x, ageDepth), num_parallel_calls=tf.data.AUTOTUNE, name="age_one_hot")
    
    # Combine Labels
    labelDs = tf.data.Dataset.zip((genderLabDs, ageLabDs), name="label_zip")
    labelDs = labelDs.map(lambda g,a: {"gender": g, "age": a}, num_parallel_calls=tf.data.AUTOTUNE, name='label_dict')
    
    # Combine Images and Labels into dataset
    ds = tf.data.Dataset.zip((imageDs, labelDs))
    
    # Configure Performance
    if(configureDs is not None):
        ds = configureDs(ds)
    
    # Pre Batch Preprocessing
    for n,f in preBatch:
        ds = ds.map(f, num_parallel_calls=tf.data.AUTOTUNE, name=n)
    
    # Batch
    if(batchSize is not None):
        ds = ds.batch(batchSize, name="ds_batch")
    
    # Post Batch Preprocessing
    for n,f in postBatch:
        ds = ds.map(f, num_parallel_calls=tf.data.AUTOTUNE, name=n)
    
    return ds

### Preprocessings

In [13]:
def trainConfigPerformance(ds):
    #ds = ds.cache()
    ds = ds.shuffle(buffer_size=100)
    ds = ds.repeat()
    return ds

In [14]:
def valConfigPerformance(ds):
    #ds = ds.cache()
    return ds

#### Preprocessing steps according to the reference paper

In [15]:
def trainPreprocessA(img, lab):
        
        img = tf.image.random_crop(img, [CROP_HEIGHT, CROP_WIDTH, 3])
        
        ud = tf.random.uniform([], dtype=tf.float32)
        udCond = tf.less(ud, 0.5)
        
        img = tf.cond(udCond, lambda: tf.image.flip_up_down(img), lambda: img)
        
        lr = tf.random.uniform([], dtype=tf.float32)
        lrCond = tf.less(lr, 0.5)
        
        img = tf.cond(lrCond, lambda: tf.image.flip_left_right(img), lambda: img)
        
        img = tf.image.random_brightness(img, 63/255)
        
        img = tf.image.random_contrast(img, 0.2, 1.8) 
        
        return img, lab

In [16]:
def valPreprocessA(img, lab):
        
        img = tf.image.crop_to_bounding_box(img,
                                            (IMG_HEIGHT-CROP_HEIGHT) // 2,
                                            (IMG_WIDTH-CROP_WIDTH) // 2,
                                            CROP_HEIGHT,
                                            CROP_WIDTH)
        
        return img, lab

In [17]:
def imageStandardize(img, lab):
    
    img = tf.image.per_image_standardization(img)
    
    return img, lab

#### Label extractor 

In [18]:
def extractGenderLabel(img, lab):
    
    lab = lab['gender']
    
    return img, lab

In [19]:
def extractAgeLabel(img, lab):
    
    lab = lab['age']
    
    return img, lab

### Generate Folds for K-Folds validation

In [20]:
def generateFoldDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, trainBatchSize, valBatchSize, trainConfigureDs=None, trainPreBatch=[], trainPostBatch=[], valConfigureDs=None, valPreBatch=[], valPostBatch=[]):
    '''
        Returns list of (train, validation) datasets
    '''
    
    N = len(foldFiles)
    
    folds = []
    
    for i in range(N):
        
        trainFiles = foldFiles[:i]
        if(i < N-1):
            trainFiles.extend(foldFiles[i+1:])
            
        valFiles = foldFiles[i]
        
        trainDs = generateDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, trainBatchSize, configureDs=trainConfigureDs, preBatch=trainPreBatch, postBatch=trainPostBatch)
        valDs = generateDs(foldFiles, imgBaseFolder, imgPrefix, genderMap, ageMap, genderDepth, ageDepth, imgWidth, imgHeight, valBatchSize, configureDs=valConfigureDs, preBatch=valPreBatch, postBatch=valPostBatch)
        
        folds.append((trainDs, valDs))
        
    return folds

In [21]:
trainPreBatch = [("train_process_a", trainPreprocessA), 
                 ("train_standardize", imageStandardize), 
                 ("train_extract_gender", extractGenderLabel)]

valPreBatch = [("val_process_a", valPreprocessA), 
               ("val_standardize", imageStandardize), 
               ("val_extract_gender", extractGenderLabel)]

In [22]:
folds = generateFoldDs(foldFiles, "adience/faces", "coarse_tilt_aligned_face", genderMap, ageMap, 2, 8, 
                       IMG_WIDTH, IMG_HEIGHT, TRAIN_BATCH_SIZE, VAL_BATCH_SIZE, 
                       trainConfigureDs=trainConfigPerformance, trainPreBatch=trainPreBatch, 
                       valConfigureDs=valConfigPerformance, valPreBatch=valPreBatch)

2022-11-07 22:05:03.836252: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 22:05:04.338998: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 626 MB memory:  -> device: 0, name: NVIDIA TITAN RTX, pci bus id: 0000:02:00.0, compute capability: 7.5


In [23]:
len(folds)

5

### Check data

In [24]:
for x,y in folds[0][0].take(1):
    pass

In [25]:
x

<tf.Tensor: shape=(4, 227, 227, 3), dtype=float32, numpy=
array([[[[-0.02438345, -0.08879495, -1.2779995 ],
         [-0.02807008, -0.09248158, -1.2816862 ],
         [-0.07603936, -0.14044984, -1.2990865 ],
         ...,
         [ 1.3476785 ,  1.3291203 ,  0.27747446],
         [ 1.3217552 ,  1.3031969 ,  0.25155115],
         [ 1.3844782 ,  1.36592   ,  0.31427518]],

        [[-0.02426042, -0.08867192, -1.2778765 ],
         [ 0.04464967, -0.01976183, -1.1783974 ],
         [-0.05166822, -0.11607972, -1.2747158 ],
         ...,
         [ 1.3809823 ,  1.362424  ,  0.3413467 ],
         [ 1.382232  ,  1.3636737 ,  0.34259745],
         [ 1.3233935 ,  1.3048352 ,  0.28375795]],

        [[ 0.49992624,  0.43551576, -0.7536898 ],
         [ 0.3764015 ,  0.31199   , -0.84664553],
         [ 0.4521661 ,  0.42090252, -0.74878216],
         ...,
         [ 1.4469931 ,  1.4284348 ,  0.4073575 ],
         [ 1.4017929 ,  1.3832346 ,  0.36215732],
         [ 1.3489764 ,  1.3304181 ,  0.3093419

In [26]:
y

<tf.Tensor: shape=(4, 2), dtype=float32, numpy=
array([[1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)>

### Model

In [28]:
def createModel():
    inp = tf.keras.Input(shape=(CROP_HEIGHT, CROP_WIDTH, 3))

    backbone = tf.keras.applications.MobileNetV3Large(include_top=False, input_shape=(CROP_HEIGHT, CROP_WIDTH, 3))
    backbone.trainable = False    
    
    flat1 = tf.keras.layers.Flatten(name='flat1')
    fc1 = tf.keras.layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.005), bias_initializer='ones', name='fc1')
    do1 = tf.keras.layers.Dropout(0.5, name='do1')

    fc2 = tf.keras.layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.005), bias_initializer='ones', name='fc2')
    do2 = tf.keras.layers.Dropout(0.5, name='do2')

    fc3 = tf.keras.layers.Dense(2, activation='softmax', kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01), name='fc3')

    o = tf.keras.applications.mobilenet_v3.preprocess_input(inp)
    o = backbone(o, training=False)
    o = flat1(o)
    o = fc1(o)
    o = do1(o)

    o = fc2(o)
    o = do2(o)

    o = fc3(o)

    model = tf.keras.models.Model(inputs=inp, outputs=o, name='baseline_1')
    
    return model

In [29]:
model = createModel()

12683000/12683000 [==============================] - 2s 0us/step


In [30]:
model.summary()

Model: "baseline_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 227, 227, 3)]     0         
                                                                 
 MobilenetV3large (Functiona  (None, 8, 8, 960)        2996352   
 l)                                                              
                                                                 
 flat1 (Flatten)             (None, 61440)             0         
                                                                 
 fc1 (Dense)                 (None, 128)               7864448   
                                                                 
 do1 (Dropout)               (None, 128)               0         
                                                                 
 fc2 (Dense)                 (None, 128)               16512     
                                                        

### Compile Model

In [31]:
STEPS_PER_EPOCH = 1000
EPOCH = 100

START_EPOCH = 0
END_EPOCH = START_EPOCH + EPOCH

#### Learning Rate Schedule

In [32]:
def schedule(epoch, lr):
    return lr

In [34]:
logBasePath = "log/baseline_mobilenet_1"
logPrefix = "log"

In [35]:
for i, (train, val) in enumerate(folds):
    
    model = createModel()
    
    opt = tf.keras.optimizers.Adam(learning_rate=0.00005)
    loss = tf.keras.losses.CategoricalCrossentropy()
    metrics = ['accuracy']
    
    model.compile(optimizer=opt, loss=loss, metrics=metrics)
    
    tboardCb = tf.keras.callbacks.TensorBoard(log_dir=f"{logBasePath}/{logPrefix}_{i}", histogram_freq=1, profile_batch=(2,10))
    lrSchedule = tf.keras.callbacks.LearningRateScheduler(schedule)
    
    cbs = [tboardCb, lrSchedule]
    
    model.evaluate(val)
    model.fit(train, initial_epoch=START_EPOCH, epochs=END_EPOCH, steps_per_epoch=STEPS_PER_EPOCH, callbacks=cbs, validation_data=val)

2022-11-07 22:07:22.562911: I tensorflow/core/profiler/lib/profiler_session.cc:99] Profiler session initializing.
2022-11-07 22:07:22.562933: I tensorflow/core/profiler/lib/profiler_session.cc:114] Profiler session started.
2022-11-07 22:07:22.562957: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1665] Profiler found 1 GPUs
2022-11-07 22:07:22.563195: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64:/home/ntcadmin/env/trt-11.0/TensorRT-7.1.3.4/lib:
2022-11-07 22:07:22.563300: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so'; dlerror: libcupti.so: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.2/lib64:/home/ntcadmin/env/trt-11.0/TensorRT-7.1.3.4/lib:
2022-11-07 22:07:22.563314: 

3528/3528 [==============================] - 49s 13ms/step - loss: 0.7092 - accuracy: 0.4768


2022-11-07 22:08:11.602696: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 372.31M (390397952 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-11-07 22:08:11.604214: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 372.31M (390397952 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-11-07 22:08:11.609488: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 372.31M (390397952 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-11-07 22:08:11.610599: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 372.31M (390397952 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-11-07 22:08:11.615961: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 372.31M (390397952 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2022-11-07 22:08:11.618162: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate

Epoch 1/100
   9/1000 [..............................] - ETA: 14s - loss: 0.6356 - accuracy: 0.7222 

2022-11-07 22:08:14.972161: I tensorflow/core/profiler/lib/profiler_session.cc:99] Profiler session initializing.
2022-11-07 22:08:14.972185: I tensorflow/core/profiler/lib/profiler_session.cc:114] Profiler session started.
2022-11-07 22:08:14.972457: E tensorflow/core/profiler/internal/gpu/cupti_error_manager.cc:133] cuptiGetTimestamp: ignored due to a previous error.
2022-11-07 22:08:14.972472: E tensorflow/core/profiler/internal/gpu/cupti_error_manager.cc:184] cuptiSubscribe: ignored due to a previous error.
2022-11-07 22:08:14.972479: E tensorflow/core/profiler/internal/gpu/cupti_error_manager.cc:457] cuptiGetResultString: ignored due to a previous error.
2022-11-07 22:08:14.972485: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1716] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error 
2022-11-07 22:08:15.101062: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-11-07 22:08

...

2022-11-08 04:45:22.755356: I tensorflow/core/profiler/lib/profiler_session.cc:99] Profiler session initializing.
2022-11-08 04:45:22.755382: I tensorflow/core/profiler/lib/profiler_session.cc:114] Profiler session started.
2022-11-08 04:45:22.755526: E tensorflow/core/profiler/internal/gpu/cupti_error_manager.cc:133] cuptiGetTimestamp: ignored due to a previous error.
2022-11-08 04:45:22.755536: E tensorflow/core/profiler/internal/gpu/cupti_error_manager.cc:184] cuptiSubscribe: ignored due to a previous error.
2022-11-08 04:45:22.755540: E tensorflow/core/profiler/internal/gpu/cupti_error_manager.cc:457] cuptiGetResultString: ignored due to a previous error.
2022-11-08 04:45:22.755544: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1716] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error 
2022-11-08 04:45:23.259836: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-11-08 04:45

  18/1000 [..............................] - ETA: 50s - loss: 0.6290 - accuracy: 0.7222 

2022-11-08 04:45:23.484949: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: log/baseline_mobilenet_1/log_4/plugins/profile/2022_11_08_04_45_23

2022-11-08 04:45:23.497508: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to log/baseline_mobilenet_1/log_4/plugins/profile/2022_11_08_04_45_23/ntcadmin-scse.memory_profile.json.gz
2022-11-08 04:45:23.498788: I tensorflow/core/profiler/rpc/client/capture_profile.cc:251] Creating directory: log/baseline_mobilenet_1/log_4/plugins/profile/2022_11_08_04_45_23
Dumped tool data for xplane.pb to log/baseline_mobilenet_1/log_4/plugins/profile/2022_11_08_04_45_23/ntcadmin-scse.xplane.pb
Dumped tool data for overview_page.pb to log/baseline_mobilenet_1/log_4/plugins/profile/2022_11_08_04_45_23/ntcadmin-scse.overview_page.pb
Dumped tool data for input_pipeline.pb to log/baseline_mobilenet_1/log_4/plugins/profile/2022_11_08_04_45_23/ntcadmin-scse.input_pipeline.

1000/1000 [==============================] - 64s 61ms/step - loss: 0.6699 - accuracy: 0.6010 - val_loss: 0.6674 - val_accuracy: 0.5960 - lr: 5.0000e-05
Epoch 2/100
1000/1000 [==============================] - 60s 60ms/step - loss: 0.6370 - accuracy: 0.6482 - val_loss: 0.6372 - val_accuracy: 0.6469 - lr: 5.0000e-05
Epoch 3/100
1000/1000 [==============================] - 60s 60ms/step - loss: 0.6112 - accuracy: 0.6543 - val_loss: 0.6136 - val_accuracy: 0.6767 - lr: 5.0000e-05
Epoch 4/100
1000/1000 [==============================] - 60s 60ms/step - loss: 0.6091 - accuracy: 0.6685 - val_loss: 0.7301 - val_accuracy: 0.5472 - lr: 5.0000e-05
Epoch 5/100
1000/1000 [==============================] - 60s 60ms/step - loss: 0.5920 - accuracy: 0.6955 - val_loss: 0.5962 - val_accuracy: 0.6907 - lr: 5.0000e-05
Epoch 6/100
1000/1000 [==============================] - 60s 61ms/step - loss: 0.5752 - accuracy: 0.7010 - val_loss: 0.5972 - val_accuracy: 0.6769 - lr: 5.0000e-05
Epoch 7/100
1000/1000 [=====

Epoch 51/100
1000/1000 [==============================] - 60s 60ms/step - loss: 0.4912 - accuracy: 0.7595 - val_loss: 0.5720 - val_accuracy: 0.7036 - lr: 5.0000e-05
Epoch 52/100
1000/1000 [==============================] - 60s 60ms/step - loss: 0.4936 - accuracy: 0.7552 - val_loss: 0.4730 - val_accuracy: 0.7658 - lr: 5.0000e-05
Epoch 53/100
1000/1000 [==============================] - 60s 61ms/step - loss: 0.5320 - accuracy: 0.7283 - val_loss: 0.4587 - val_accuracy: 0.7780 - lr: 5.0000e-05
Epoch 54/100
1000/1000 [==============================] - 60s 60ms/step - loss: 0.4605 - accuracy: 0.7815 - val_loss: 0.5675 - val_accuracy: 0.7311 - lr: 5.0000e-05
Epoch 55/100
1000/1000 [==============================] - 60s 60ms/step - loss: 0.5212 - accuracy: 0.7465 - val_loss: 0.4808 - val_accuracy: 0.7647 - lr: 5.0000e-05
Epoch 56/100
1000/1000 [==============================] - 60s 60ms/step - loss: 0.5007 - accuracy: 0.7523 - val_loss: 0.5673 - val_accuracy: 0.6877 - lr: 5.0000e-05
Epoch 57/1